# Peafowl Creator

Use this page to create peafowl and visualize the results of breeding pairs, and to see the possible genetic makeup of various peafowl

In [89]:
import io
import os.path
import ipywidgets as widgets
from PIL import Image

Use the following widget to add unique features to a peafowl, and select the peafowl's sex.  Any features that are not added to the peafowl will be assumed to be wild type

In [90]:
# Define color genes
colorGenes = [
    # Note: the "default" gene aka Wild Type is not included because it is a special
    # case that can apply to alleles of any gene
    { 'notation': 'br',      'name': 'Bronze' },
    { 'notation': 'o',       'name': 'Opal' },
    { 'notation': 'md',      'name': 'Midnight' },
    { 'notation': 'j',       'name': 'Jade' },
    { 'notation': 'mo',      'name': 'Montana' },
    { 'notation': 'ch',      'name': 'Charcoal' },
    { 'notation': 'st',      'name': 'Steel' },
    { 'notation': 'um',      'name': 'Ultramarine' },
    { 'notation': 'bu',      'name': 'Burnt Umber' }
]

sexLinkedColorAllotypes = [
    { 'notation': 'Z(c)',    'name': 'Cameo' },
    { 'notation': 'Z(pl)',   'name': 'American Purple' },
    { 'notation': 'Z(va)',   'name': 'Sonja\'s Violet' },
    { 'notation': 'Z(ve)',   'name': 'European Violet' },
    # Note: even though peach is actually 2 genes, we are treating
    # it as one for the sake of this code, and dealing with het Peach
    # as a special phenotype
    { 'notation': 'Z(pl:c)', 'name': 'Peach' }
]

# Define what genes combinations form special colors
multiGeneColors = [
    { 'name': 'Platinum',      'genes': ['br', 'o']}
]

sexAndAutosomalComboColors = [
    { 'name': 'Taupe',         'autosomalGene': 'o',  'sexGene': 'Z(pl)' },
    { 'name': 'Mocha',         'autosomalGene': 'md', 'sexGene': 'Z(pl)' },
    { 'name': 'Ivory',         'autosomalGene': 'o',  'sexGene': 'Z(c)' },
    { 'name': 'Indigo',        'autosomalGene': 'br', 'sexGene': 'Z(pl)' },
    { 'name': 'Hazel',         'autosomalGene': 'br', 'sexGene': 'Z(pl)' }
]

hetSexColors = [
    { 'name': 'Midway between Violet and Purple', 'alleles': ['Z(pl)', 'Z(ve)'] }
]

# Define pattern genes
patternGenes = [
    { 'notation': 'bs',      'name': 'Blackshoulder' }
]

# Define leucistic genes
leucisticGenes = [
    { 'notation': 'p',       'name': 'Pied' },
    { 'notation': 'WE',       'name': 'White Eye' }
]

piedAllotypes = [
    { 'notation': 'p',       'name': 'Dark Pied' },
    { 'notation': 'W',       'name': 'White' }
]

whiteEyeAllotypes = [
    { 'notation': 'WE',      'name': 'White Eye' },
    { 'notation': 'sWE',     'name': 'Silver White Eye' }
]

# Define lucistic special cases
hetLeucistic = [
    { 'name': 'Pied', 'alleles': ['W', 'p'] }
]

If you want to look at more complex pairings, including outcrosses, you can also edit the bird's geneotype below. Changes to the genotype will also change the phenotype inputs, so that both types of inputs show the same bird.

In [91]:
# These functions will let us translate between genotypes and phenotypes
def saveGenotypeToBird(bird, geneName, genotype):
    bird[geneName] = genotype

In [92]:
# This converts a phenotype to a genotype in a bird
# assuming the phenotype breeds true
def savePhenotypeToBird(bird, phenotype, sex, simpleGenes = colorGenes, sexLinkedTrait = 'Sex-Linked Color', 
                        sexLinkedAllotypes = sexLinkedColorAllotypes, multiGeneTraits = multiGeneColors, 
                        sexAndAutosomalCombos = sexAndAutosomalComboColors, hetSexTraits = hetSexColors):
    foundPhenotype = False
    
    # Check normal trait genes
    for gene in simpleGenes:
        # If the current gene we are checking matches the selected
        # phenotype, set the genotype in the bird equal to homozygous
        # for the trait
        if phenotype == gene['name']:
            bird[gene['name']] = gene['notation'] + '/' + gene['notation']
            foundPhenotype = True
        # If the current gene we are checking does not match the phenotype
        # we should clear the genotype for that gene
        elif gene['name'] in bird:
            del bird[gene['name']]

    # Reset sex-linked traits to default wild type
    if sexLinkedTrait in bird:
        del bird[sexLinkedTrait]
        
    # Check all sex-linked allotypes the same way we checked the autosomal
    # genes
    for allotype in sexLinkedAllotypes:
        if phenotype == allotype['name']:
            if sex == 'Male':
                bird[sexLinkedTrait] = allotype['notation'] + '/' + allotype['notation']
            else: 
                bird[sexLinkedTrait] = allotype['notation'] + '/w'
            foundPhenotype = True

    # At this point, all trait genes have been cleared
    # except for a matched phenotype
    # If we have found the matching phenotype already,
    # we can stop here
    if foundPhenotype:
        return

    # Save the phenotype data for genotypes that match multiple genes
    for trait in multiGeneTraits:
        if phenotype == trait['name']:
            # for each gene that makes up the multi-gene trait
            for traitGeneNotation in trait['genes']:
                # find that gene in the gene list and save it to the bird
                for gene in simpleGenes:
                    if gene['notation'] == traitGeneNotation:
                        bird[gene['name']] = gene['notation'] + '/' + gene['notation']

    # Save the phoenotype data for genes that require both autosomal and sex-linked trait
    for trait in sexAndAutosomalCombos:
        if phenotype == trait['name']:
            # Save autosomal phenotype
            for gene in simpleGenes:
                if gene['notation'] == trait['autosomalGene']:
                    bird[gene['name']] = gene['notation'] + '/' + gene['notation']

            # Save sex phenotype
            for allotype in sexLinkedAllotypes:
                if allotype['notation'] == trait['sexGene']:
                    if sex == 'Male':
                        bird[sexLinkedTrait] = allotype['notation'] + '/' + allotype['notation']
                    else: 
                        bird[sexLinkedTrait] = allotype['notation'] + '/w'
                        
    # Save phenotype for het sex traits
    for trait in hetSexTraits:
        if phenotype == trait['name'] and sex == 'Male':
            bird[sexLinkedTrait] = trait['alleles'][0] + '/' + trait['alleles'][1]

In [93]:
def getColorFromBird(bird, sex):
    # Birds are wild type by default
    finalBirdColor = "Wild Type"
    # Because colors are not mutually exclusive, we want to track 
    # all possible colors
    otherBirdColors = []
    
    # Set sex to default if not defined so we don't 
    # break things when we try to check the sex-linked color
    if not 'Sex-Linked Color' in bird:
        if sex == 'Male':
            bird['Sex-Linked Color'] = 'Z(WT)/Z(WT)'
        else: 
            bird['Sex-Linked Color'] = 'Z(WT)/w'
    
    # Bird color special cases/Documents/AccessAdminAssist/access-admin-assist-frontend
    if bird['Sex-Linked Color'] == 'Z(c)/Z(pl:c)':
        finalBirdColor = 'Cameo'
    elif bird['Sex-Linked Color'] == 'Z(pl)/Z(pl:c)':
        finalBirdColor = 'American Purple'

    # Check if bird color matches any simple colors
    for gene in colorGenes:
        if gene['name'] in bird:
            if bird[gene['name']] == gene['notation'] + '/' + gene['notation']:
                # Add the previous bird color to a list, and change the final color
                otherBirdColors.append(finalBirdColor)
                finalBirdColor = gene['name']

    # Check if bird color matches any sex-linked colors
    for allotype in sexLinkedColorAllotypes:
        if 'Sex-Linked Color' in bird and ((sex == 'Male' and bird['Sex-Linked Color'] == allotype['notation'] + '/' + allotype['notation']) or
            (sex == 'Female' and bird['Sex-Linked Color'] == allotype['notation'] + '/w')):
            # Add the previous bird color to a list, and change the final color
            otherBirdColors.append(finalBirdColor)
            finalBirdColor = allotype['name']

    # Check bird color matches sexAndAutosomal Combo
    for color in sexAndAutosomalComboColors:
        matchesAll = True
        
        for gene in colorGenes:
            # If the current gene is part of the color
            if (gene['notation'] == color['autosomalGene'] and 
                # and the current gene is NOT in the bird
                (not gene['name'] in bird or 
                bird[gene['name']] != gene['notation'] + '/' + gene['notation'])):
                # then we know the bird does NOT have the color, and we do not 
                # have to check any other genes
                matchesAll = False
                break

        # If we did not find an autosomal gene, we don't need to check the
        # sex-linked genes
        if not matchesAll:
            continue
        
        for allotype in sexLinkedColorAllotypes:
            # If the current gene is part of the color
            if (allotype['notation'] == color['sexGene'] and 
                # and the current gene is NOT in the bird
                (not 'Sex-Linked Color' in bird or 
                 (sex == 'Male' and bird['Sex-Linked Color'] != allotype['notation'] + '/' + allotype['notation']) or
                 (sex == 'Female' and bird['Sex-Linked Color'] != allotype['notation'] + '/w'))):
                matchesAll = False
                break

        # If matchesAllGenes is still true here, we know the bird must have
            # all the needed genes for the color
        if matchesAll:
            # Add the previous bird color to a list, and change the final color
            otherBirdColors.append(finalBirdColor)
            finalBirdColor = color['name']
                

    # Check bird color matches multi-gene color
    for color in multiGeneColors:
        # Make sure all genes needed to make the color are availible in the bird
        matchesAllGenes = True
        for gene in colorGenes:
            # If the current gene is part of the color
            if (gene['notation'] in color['genes'] and 
                # and the current gene is NOT in the bird
                ( not gene['name'] in bird or 
                bird[gene['name']] != gene['notation'] + '/' + gene['notation'])):
                # then we know the bird does NOT have the color, and we do not 
                # have to check any other genes
                matchesAllGenes = False
                break

        # If matchesAllGenes is still true here, we know the bird must have
        # all the needed genes for the color
        if matchesAllGenes:
            # Add the previous bird color to a list, and change the final color
            otherBirdColors.append(finalBirdColor)
            finalBirdColor = color['name']

    for color in hetSexColors:
        if bird['Sex-Linked Color'] == color['alleles'][0] + '/' + color['alleles'][1]:
            # Add the previous bird color to a list, and change the final color
            otherBirdColors.append(finalBirdColor)
            finalBirdColor = color['name']

    return (finalBirdColor, otherBirdColors)

In [94]:
# This function creates a handful of widgets that
# allow users to select the phenotype of a bird
# If we then assume all traits breed true, we
# can generate a genotype without making the
# user understand bird genetics
def createPeacockPhenotypeWidgets(peacockName):
    colorList = (['Wild Type'] + 
        # TECH EXPLANATION
        # lambda indicates an inline function
        # map() means that the lambda function is called for each item in the colorGenes array
        # It converts each gene's dictionary into just the name column
        # list() turns that map back into an array
        # The goal here is to make sure all genes show up as human readable options 
        # in the list, and they don't get out of sync with each other
        list(map(lambda gene: gene['name'], colorGenes)) + 
        list(map(lambda allotype: allotype['name'], sexLinkedColorAllotypes)) +
        list(map(lambda phenotype: phenotype['name'], multiGeneColors)) +
        list(map(lambda phenotype: phenotype['name'], sexAndAutosomalComboColors)) +
        list(map(lambda phenotype: phenotype['name'], hetSexColors)))
        
    
    color = widgets.Dropdown(
        options=colorList,
        value=colorList[0],
        description='Color:',
        disabled=False,
    )

    # See colorList for technical explanation
    patternList = (['Barred Wing Wild Type'] +
        list(map(lambda gene: gene['name'], patternGenes)))
    pattern = widgets.Dropdown(
        options=patternList,
        value=patternList[0],
        description='Pattern:',
        disabled=False,
    )
    
    # See colorList for technical explanation
    leucisticList = (['Non-Leucistic Wild Type'] +
       list(map(lambda allotype: allotype['name'], piedAllotypes)) +
       list(map(lambda allotype: allotype['name'], whiteEyeAllotypes)) +
       list(map(lambda phenotype: phenotype['name'], hetLeucistic)))
    
    leucistic = widgets.Dropdown(
        options=leucisticList,
        value=leucisticList[0],
        description='Leucistic:',
        disabled=False,
    )

    return (
        widgets.VBox([
            widgets.Label(value=peacockName),
            color, pattern, leucistic
        ]),
        {
            'color': color,
            'pattern': pattern,
            'leucistic': leucistic
        }
    )

In [106]:
# This lets us turn a single gene into a genotype widget
# We will run this function over all the autosomal color genes
# and the shoulder genes to generate appropriate widgets
def createGenotypeWidgetFromGene(gene):
    possibilities = [
        'WT/WT', 
        'WT/' + gene['notation'],
        gene['notation'] + '/' + gene['notation']
    ]
    return widgets.Dropdown(
        options=possibilities,
        value=possibilities[0],
        description=gene['name'] + ':',
        disabled=False,
    )

# For sex-linked genes, we want to change the genotype based on sex,
# and treat different allotypes as mutually exclusive
def createSexLinkedGeneWidget(allotypes, sex):
    allotypesWithWild = [{ 'notation': 'Z(WT)',    'name': 'Wild Type' }] + allotypes
    possibilities = []

    # Iterate through each allotype as a possible first allele.
    # In this case, we use enumerate instead of a normal for each loop so we can limit
    # the loop of the second allele and prevent duplicate genotypes
    for index, firstAllele in enumerate(allotypesWithWild):
        if sex == 'Male':
            # For males, iterate through each allotype starting with the firstAllele as a possible second allele
            # This gives us every possible combination of alleles without duplicates 
            # You can think of it as the top trianlge of a possibilities square
            # The [index:] selects all items in the list starting with the same item as the current firstAllele
            for secondAllele in allotypesWithWild[index:]:
                possibilities.append(firstAllele['notation'] + '/' + secondAllele['notation'])
        else:
            # For females just add the missing sex chromasome 'w' to the first allele
            possibilities.append(firstAllele['notation'] + '/w');
    
    return widgets.Dropdown(
        options=possibilities,
        value=possibilities[0],
        description='Sex-Linked Color:',
        disabled=False,
    )

# Used for troubleshooting bugs with print()
output = widgets.Output()

# When genotype changes, update the related phenotype
# TODO - add input arrays to saveGenotypeToBird
def handleGenotypeChange(phenotypeWidget, bird, sex, geneName, newGenotype):
    with output:
        saveGenotypeToBird(bird, geneName, newGenotype)
        (phenotype, prevPhenotypes) = getColorFromBird(bird, sex)

        # Set the index and the value of the phenotype widget to match the new genotype
        # We have to set both, or else the display won't update properly
        phenotypeWidget.index = phenotypeWidget.options.index(phenotype)
        phenotypeWidget.value = phenotype

def handlePhenotypeColorChange(thisWidget, genotypeWidgets, bird, sex, phenotype):
    with output:
        savePhenotypeToBird(bird, phenotype, sex)
        # We may need to change multiple genotype widgets in order to match the 
        # current phenotype.
        # We don't want the phenotype widget to be altered by half-finished changes, so we
        # need to temorarily prevent the phenotype from changing when the genotype changes
        thisWidget.unobserve(None, 'value')

        # Then we need to change all the relevant genotype widgets
        for widget in genotypeWidgets:
            if widget.description[:-1] in bird:
                widget.value = bird[widget.description[:-1]]
            else:
                widget.value = widget.options[0]

        # Once all changes are made, we need to re-build the observer
        thisWidget.observe(lambda changedValue: handlePhenotypeColorChange(thisWidget, genotypeWidgets, bird, sex, changedValue['new']), 'value')

def createPeacockGenotypeWidgets(peacockName, sex, phenotypeWidgets):
    bird = {}
    
    # For each possible color, create a widget with genotype options
    colors = []
    for gene in colorGenes:
        widget = createGenotypeWidgetFromGene(gene)
        colors.append(widget)
        # This updates the color phenotype whenever any color genotype changes
        widget.observe(
            lambda changedValue, geneName=gene['name']: 
                handleGenotypeChange(phenotypeWidgets['color'], bird, sex, geneName, changedValue['new']), 
            'value')

    # Create a single widget for all the sex-linked color varients
    sexLinkedColor = createSexLinkedGeneWidget(sexLinkedColorAllotypes, sex)
    # This updates the phenotype whenever the sex-linked color genotype changes
    sexLinkedColor.observe(
        lambda changedValue, sex=sex: 
            handleGenotypeChange(phenotypeWidgets['color'], bird, sex, 'Sex-Linked Color', changedValue['new']), 
        'value')

    # This updates the genotype(s) when the phenotype changes
    phenotypeWidgets['color'].observe(
        lambda changedValue: 
            handlePhenotypeColorChange(phenotypeWidgets['color'], colors + [sexLinkedColor], bird, sex, changedValue['new']), 
        'value')

    return (
        widgets.VBox([widgets.Label(value=peacockName)] + colors + [sexLinkedColor]),
        {
            'colors': colors,
            'sexLinkedColor': sexLinkedColor
        }
    )

In [ ]:
def generatePeafowlImage(sex, color):
    # Color should be the base layer
    colorImageName = "Images/" + sex + "/Color/" + color + ".png"
    if not os.path.isfile(colorImageName):
        # Default to the unknown color if we don't have an image
        # for the defined color
        colorImageName = "Images/" + sex + "/Color/Unknown.png"

    # Open the file for the color image
    peafowlColor = Image.open(colorImageName)
    # Open the lineart image
    peafowlLineart = Image.open("Images/" + sex + "/lineart.png")

    # Merge the images as layers
    peafowlColor.paste(peafowlLineart, (0, 0), peafowlLineart)

    # Extract the image data as bytes
    imageData = io.BytesIO()
    peafowlColor.save(imageData, format='PNG')

    # Return a widget-compatible image
    return imageData

In [119]:
# This actually displays the UI that is set up in previous code blocks

# Create the peafowl images, but do not display them yet
maleImage = widgets.Image(
    value=generatePeafowlImage("Male", "Unknown").getvalue(),
    format='png',
    width=300,
    height=300,
)
femaleImage = widgets.Image(
    value=generatePeafowlImage("Female", "Unknown").getvalue(),
    format='png',
    width=300,
    height=300,
)

# Display the phenotype inputs
display(widgets.Label(value="Enter phsyical traits for a breeding pair of peacocks. Note: Traits with multiple genotypes will be assumed to be homozygous ie breed true"))
(maleWidgetsBox, maleWidgets) = createPeacockPhenotypeWidgets("Male Peacock")
(femaleWidgetsBox, femaleWidgets) = createPeacockPhenotypeWidgets("Female Peacock")
display(widgets.HBox([femaleWidgetsBox, maleWidgetsBox]))

# Display the genotype inputs
display(widgets.Label(value="Or edit the genotype of the birds directly:"))
(maleWidgets, maleGenotypeData) = createPeacockGenotypeWidgets("Male Peacock", 'Male', maleWidgets)
(femaleWidgets, femaleGenotypeData) = createPeacockGenotypeWidgets("Female Peacock", 'Female', femaleWidgets)
display(widgets.HBox([femaleWidgets, maleWidgets]))
display(output)

# Display the images last
display(widgets.HBox([femaleImage, maleImage]))

Label(value='Enter phsyical traits for a breeding pair of peacocks. Note: Traits with multiple genotypes will …

Label(value='Or edit the genotype of the birds directly:')

Output()